
## 1. What is an AI Agent?

### Core Definition
An **AI agent** is an autonomous system that uses a language model (LM) to iteratively reason about problems, make decisions, and take actions in pursuit of a goal. Unlike simple chatbots that respond to single queries, agents can:

- Break down complex tasks into subtasks
- Make multi-step plans
- Use external tools and APIs
- Gather information dynamically
- Adapt based on feedback
- Operate with varying degrees of autonomy

### Key Characteristics

**Autonomy**: Agents can operate independently without constant human intervention, making decisions based on their observations and goals.

**Reactivity**: They perceive their environment (through APIs, databases, search results, etc.) and respond to changes in it.

**Proactivity**: Agents take initiative to achieve goals, not just responding to immediate stimuli.

**Goal-oriented behavior**: All actions are directed toward achieving specific objectives, whether explicit (user-defined) or implicit (system-designed).

### Agent vs. Traditional LLM Applications

| Traditional LLM | AI Agent |
|----------------|----------|
| Single-turn responses | Multi-turn interactions with memory |
| Static context | Dynamic environment interaction |
| No external actions | Tool usage and API calls |
| Passive information retrieval | Active problem-solving |
| Deterministic flow | Adaptive decision-making |

---

## 2. Reasoning vs. Action

This is a fundamental terminolgy in agent design that addresses **when to think** versus **when to act**.

### Reasoning (Thought)

**Definition**: Internal cognitive processes where the agent analyzes information, plans, and deliberates without taking external actions.

**Characteristics**:
- Happens "in the agent's head" (within the LM's context)
- No side effects on the environment
- Can involve chain-of-thought (CoT), planning, reflection
- Consumes tokens but doesn't execute actions

**Examples of reasoning**:
```
"Let me think about this step by step..."
"To solve this, I need to first understand X, then Y..."
"The user wants A, but to get A, I need to gather B and C first..."
```

**Advantages**:
- Reduces expensive/irreversible actions
- Enables planning and foresight
- Allows error detection before execution
- Supports complex problem decomposition

**Disadvantages**:
- Can lead to "analysis paralysis"
- May hallucinate about environment state without verification
- Consumes context window without gathering real information

### Action (Execution)

**Definition**: External operations that interact with and modify the environment or gather real-world information.

**Characteristics**:
- Observable effects outside the agent
- Tool calls, API requests, database queries
- Can have side effects (writing files, sending emails)
- Provides ground truth information

**Examples of actions**:
```python
# Tool calls
search_web(query="current weather in Tokyo")
execute_code(code="import pandas as pd...")
send_email(to="user@example.com", subject="Report")
write_file(path="/data/output.json", content="{...}")
```

**Advantages**:
- Grounds reasoning in reality
- Enables verification of hypotheses
- Necessary for task completion
- Provides real-time information

**Disadvantages**:
- Can be expensive (API costs, latency)
- May have irreversible consequences
- Requires careful error handling
- Potential for unintended side effects

### The Balance: Interleaving Reasoning and Action

Modern agents don't choose one or the other—they **interleave** reasoning and action in a loop:

1. **Observe** the current state
2. **Reason** about what to do next
3. **Act** (if necessary) to gather information or make progress
4. **Observe** the result of the action
5. Repeat until goal is achieved

This creates a **perception-cognition-action cycle** similar to human problem-solving.

---

## 3. ReAct: Reasoning + Acting Paradigm

### What is ReAct?

**ReAct** (Reasoning and Acting) is a seminal framework introduced by Yao et al. (2023) that synergizes reasoning traces and task-specific actions in language models. It's one of the most influential agent architectures.

**Paper**: "ReAct: Synergizing Reasoning and Acting in Language Models" (ICLR 2023)

### Core Concept

ReAct prompts the LM to generate both:
- **Reasoning traces** (Thought): Internal dialogue explaining the agent's thinking
- **Actions**: Specific tool calls or operations to execute

These are interleaved in a structured format, creating a reasoning-action-observation loop.

### ReAct Structure

```
Question: [User's question]

Thought 1: [Agent's reasoning about what to do first]
Action 1: [Specific action to take]
Observation 1: [Result of Action 1]

Thought 2: [Agent's reasoning given the observation]
Action 2: [Next action based on new information]
Observation 2: [Result of Action 2]

...

Thought N: [Final reasoning]
Answer: [Final answer to the question]
```

### Example: ReAct in Action

**Question**: What is the elevation of the highest peak in the state where the University of Washington is located?

```
Thought 1: I need to find out what state the University of Washington is in.
Action 1: Search[University of Washington location]
Observation 1: The University of Washington is located in Seattle, Washington.

Thought 2: Now I know it's in Washington state. I need to find the highest peak in Washington.
Action 2: Search[highest peak in Washington state]
Observation 2: The highest peak in Washington state is Mount Rainier.

Thought 3: Now I need to find the elevation of Mount Rainier.
Action 3: Search[Mount Rainier elevation]
Observation 3: Mount Rainier has an elevation of 14,411 feet (4,392 meters).

Thought 4: I have all the information needed to answer the question.
Answer: The elevation of the highest peak in Washington state (Mount Rainier) is 14,411 feet or 4,392 meters.
```

### Why ReAct Works

**Explicit reasoning improves action selection**: By forcing the model to articulate its thinking, it makes better decisions about which tools to use and when.

**Actions ground reasoning in reality**: Instead of hallucinating facts, the agent retrieves real information, reducing errors.

**Interpretability**: The thought traces make the agent's decision-making process transparent and debuggable.

**Error recovery**: When an action fails or returns unexpected results, the reasoning step helps the agent adapt and try alternative approaches.

**Human-like problem solving**: Mirrors how humans think-aloud while solving problems, alternating between planning and execution.

### ReAct Variants and Extensions

**ReAct + Self-Reflection**: Agent reviews its own reasoning and actions to identify and correct mistakes.

**ReAct + Memory**: Stores past reasoning-action trajectories to learn from experience.

**Reflexion**: Extends ReAct with explicit self-reflection and learning from failures.

**Tree of Thoughts (ToT)**: Explores multiple reasoning paths simultaneously, using search algorithms.

---

## 4. Tool-Calling Behavior

### What is Tool Calling?

**Tool calling** (also called function calling) is the mechanism by which an LLM invokes external functions, APIs, or capabilities beyond text generation. It's the primary way agents interact with the outside world.

### How Tool Calling Works

#### 1. Tool Definition

Tools are defined with schemas that specify:
- **Name**: Identifier for the tool
- **Description**: What the tool does (critical for selection)
- **Parameters**: Expected inputs with types and descriptions
- **Return type**: What the tool outputs

**Example Tool Schema**:
```json
{
  "name": "web_search",
  "description": "Search the web for current information on a topic",
  "parameters": {
    "type": "object",
    "properties": {
      "query": {
        "type": "string",
        "description": "The search query to execute"
      },
      "num_results": {
        "type": "integer",
        "description": "Number of results to return (default: 5)"
      }
    },
    "required": ["query"]
  }
}
```

#### 2. Tool Selection

The LM analyzes the user's request and available tools to determine:
- **Which tool(s)** to use
- **What parameters** to pass
- **In what order** to call them (if multiple tools needed)

This happens through few-shot prompting or fine-tuning.

#### 3. Tool Execution

The agent outputs a structured tool call (often JSON), which the orchestration layer:
- Parses and validates
- Executes against the actual tool/API
- Captures the result

#### 4. Result Integration

The tool's output is fed back to the LM as an **observation**, which then:
- Reasons about the result
- Decides next actions
- Continues until task completion

### Tool Calling Formats

Different platforms use different formats:

**OpenAI Function Calling**:
```json
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_weather",
    "arguments": "{\"location\": \"Tokyo\", \"unit\": \"celsius\"}"
  }
}
```

**Anthropic Tool Use**:
```json
{
  "type": "tool_use",
  "id": "tool_abc123",
  "name": "get_weather",
  "input": {
    "location": "Tokyo",
    "unit": "celsius"
  }
}
```

**Generic Text-Based**:
```
Action: web_search
Action Input: {"query": "Tokyo weather today"}
```

### Types of Tools

**Information Retrieval**:
- Web search (Google, Bing)
- Database queries (SQL, vector databases)
- Document retrieval (RAG systems)
- API data fetching

**Computation**:
- Code execution (Python, JavaScript)
- Mathematical calculations
- Data analysis and visualization
- Model inference (calling other ML models)

**External Actions**:
- File system operations (read/write)
- Email sending
- Calendar management
- API integrations (Slack, GitHub, etc.)

**Specialized Domain Tools**:
- Medical databases
- Financial market data
- Scientific computation libraries
- Geographic information systems

### Tool Calling Challenges

**Tool Selection Errors**: The LM might choose the wrong tool for a task, especially when multiple similar tools exist.

**Parameter Hallucination**: The model might generate plausible-looking but incorrect parameters (e.g., non-existent API endpoints).

**Error Handling**: Tools can fail due to network issues, rate limits, invalid inputs, or missing permissions. Agents must gracefully handle these.

**Context Window Limitations**: Each tool call and result consumes tokens. Long agent runs can exceed context limits.

**Execution Safety**: Some tools (like code execution or file deletion) can have dangerous side effects if misused.

### Best Practices for Tool Calling

**Clear, Specific Descriptions**: Tool descriptions should be detailed enough for the LM to understand exactly when and how to use them.

**Robust Parameter Validation**: Always validate tool inputs before execution to prevent errors or security issues.

**Meaningful Error Messages**: When tools fail, return informative error messages that help the agent recover.

**Rate Limiting and Quotas**: Implement safeguards to prevent excessive tool usage.

**Observability**: Log all tool calls and results for debugging and monitoring.

**Graceful Degradation**: Design agents to continue functioning even when some tools are unavailable.

**Security Sandboxing**: Isolate tool execution environments to prevent unauthorized access or damage.

### Advanced Tool Calling Patterns

**Parallel Tool Calling**: Execute multiple independent tool calls simultaneously to reduce latency.

**Conditional Tool Chains**: Tool B is called only if Tool A returns specific results.

**Tool Composition**: Combining outputs from multiple tools to achieve complex goals.

**Agentic Tool Creation**: Meta-agents that can define and register new tools dynamically.

**Human-in-the-Loop**: Critical tools that require human approval before execution.

---

## 5. Putting It All Together: Agent Architectures

### Simple Reflex Agent
- No reasoning, direct stimulus-response
- Fast but inflexible
- Example: Rule-based chatbot

### Model-Based Reflex Agent
- Maintains internal state/memory
- Still reactive, but context-aware
- Example: Stateful conversation bot

### Goal-Based Agent (ReAct Pattern)
- Explicit goals and planning
- Reasoning + action loops
- Example: Research assistant, task automation

### Utility-Based Agent
- Optimizes for specific metrics
- Evaluates action quality
- Example: Recommendation systems, trading bots

### Learning Agent
- Improves over time from experience
- Self-reflection and adaptation
- Example: Reflexion, reinforcement learning agents

---

## 6. Key Takeaways for Interviews

### Conceptual Understanding
- **Define agents clearly**: Autonomous, goal-oriented, use reasoning + actions
- **Explain the reasoning-action dichotomy**: Why both are necessary
- **Describe ReAct's innovation**: Interleaving thoughts and actions improves performance
- **Understand tool calling mechanics**: Schema → Selection → Execution → Integration

### Practical Skills
- **Implement a basic ReAct loop** in code
- **Design tool schemas** that are clear and unambiguous
- **Handle tool errors** gracefully with retry logic
- **Optimize for context efficiency** (minimize unnecessary reasoning/actions)

### Advanced Topics
- **Multi-agent systems**: Agents collaborating or competing
- **Agent evaluation**: How to measure success (task completion, efficiency, correctness)
- **Safety and alignment**: Preventing harmful actions, ensuring goal alignment
- **Scalability challenges**: Managing long conversations, expensive tools

### Common Interview Questions

1. **"How would you build an agent to book a restaurant reservation?"**
   - Define goal, identify needed tools (search, availability check, booking API)
   - Design ReAct loop with error handling
   - Discuss edge cases (no availability, ambiguous requests)

2. **"What's the difference between RAG and an agent?"**
   - RAG: Retrieval augmentation for single-turn QA
   - Agent: Multi-turn, autonomous, can use RAG as one tool among many

3. **"How do you prevent an agent from getting stuck in loops?"**
   - Max iteration limits
   - Detect repeated actions/states
   - Explicit termination conditions

4. **"What's the biggest challenge in agent systems?"**
   - Reliability (hallucinations, tool errors)
   - Cost (many LLM calls + tool usage)
   - Evaluation (measuring true task success)

---

In [2]:
# Reasoning vs Action using LangChain
# pip install -U langchain langchain-openai

import os
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv

load_dotenv(override=True)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ---- Reasoning step ----
reason_prompt = ChatPromptTemplate.from_template("""
You are an agent.
Observation: {observation}

Decide the next action ONLY from:
- RETRY
- ESCALATE
- IGNORE
""")

reason_chain = reason_prompt | llm

# ---- Action step ----
def take_action(action: str):
    return f"Executing action: {action}"

action_chain = RunnableLambda(take_action)

# ---- Agent loop ----
def agent(observation: str):
    decision = reason_chain.invoke({"observation": observation}).content.strip()
    return action_chain.invoke(decision)

print(agent("Build failed due to timeout"))


Executing action: RETRY


In [3]:
print(agent("Build completed successfully"))


Executing action: IGNORE


In [4]:
# ReAct style demo
# pip install -U langchain langchain-openai

import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_template("""
You are an agent using ReAct.

Observation: {observation}

Think about what to do.
Then respond in this format:

Thought: <reasoning>
Action: <action>
""")

response = llm.invoke(
    prompt.format_prompt(observation="UI test failed with element click intercepted")
)

print(response.content)


Thought: The error "element click intercepted" typically indicates that the element intended for interaction is obscured by another element or is not in a clickable state. I need to investigate the UI to determine what might be blocking the click and ensure the element is interactable.

Action: Check the visibility and position of the element in the UI, and if necessary, wait for any overlays or animations to complete before attempting the click again. If an overlay is present, I will close it or wait for it to disappear.


In [5]:
# Tool Calling Example
# pip install -U langchain langchain-openai

import os
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# ---- Define tools ----
@tool
def retry_test():
    """Retry the failed test"""
    return "Test retried successfully"

@tool
def escalate_issue():
    """Escalate issue to human"""
    return "Issue escalated to DevOps"

# ---- LLM with tools ----
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_with_tools = llm.bind_tools([retry_test, escalate_issue])

# ---- Invoke agent ----
response = llm_with_tools.invoke(
    "Test failed due to timeout. What should you do?"
)

print("LLM response:", response)
print("Tool calls:", response.tool_calls)


LLM response: content='When a test fails due to a timeout, you can take the following actions:\n\n1. **Retry the Test**: Sometimes, the failure may be temporary, and retrying the test can lead to a successful outcome.\n\n2. **Escalate the Issue**: If the test continues to fail after multiple retries, it may indicate a deeper issue that needs to be addressed by a human or a more experienced team member.\n\nWould you like me to retry the test or escalate the issue?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 99, 'prompt_tokens': 62, 'total_tokens': 161, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CzpB4IVGS1GOjc6ca2iV2c5vS7c0h', 'service_tier': 'default', 'fini

LLM response: content='When a test fails due to a timeout, you can take the following actions:\n\n1. 

**Retry the Test**: Sometimes, the failure may be temporary, and retrying the test can lead to a successful outcome.\n\n2. 

**Escalate the Issue**: If the test continues to fail after multiple retries, it may indicate a deeper issue that needs to be addressed by a human or a more experienced team member.\n\n

Would you like me to retry the test or escalate the issue?' 

additional_kwargs={'refusal': None} 

response_metadata={'token_usage': {'completion_tokens': 99, 'prompt_tokens': 62, 'total_tokens': 161, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CzpB4IVGS1GOjc6ca2iV2c5vS7c0h', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019bd7b8-bd99-7840-894f-259a22c6b4a9-0' 

tool_calls=[] 

invalid_tool_calls=[] 

usage_metadata={'input_tokens': 62, 'output_tokens': 99, 'total_tokens': 161, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}

Tool calls: []
